# Проектная работа
# Оценка потребительской лояльности (NPS) телекоммуникационной компании


# Описание проекта

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

<strong>Задача:</strong> определить текущий уровень потребительской лояльности (NPS).



## План проекта

### Шаг 1. Подключение к базе. 
Получить доступ к базе данных. Данные представлены базой данных SQLite, одним файлом.

### Шаг 2. Выгрузка данных. 
Подготовить данные для дашборда. Преобразовывать данные. 

### Шаг 3. Создание дашборда в Tableau. 
Построить дашборд, который представит информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков. 

### Шаг 4. Ответьте на вопросы с помощью дашборда
Используя дашборд, ответить на вопросы:
1. Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
2. Какие группы пользователей наиболее лояльны к сервису? Какие менее?
3. Какой общий NPS среди всех опрошенных?
4. Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

## Ход работы:

In [1]:
#Шаг 1:

In [2]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [3]:
path_to_db_platform = 'C:/Users/System_10/Desktop/DA+/Python Projects/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [4]:
#Шаг 2:

In [5]:
query = """
    SELECT a.user_id,
            a.lt_day,
            CASE 
               WHEN a.lt_day <= 365 THEN 'new'
               ELSE 'old'
            END AS is_new,
            a.age,
            CASE 
               WHEN a.gender_segment = 1 THEN 'female'
               WHEN a.gender_segment = 0 THEN 'male'
               ELSE 'other'
            END AS gender_segment,
            a.os_name,
            a.cpe_type_name,
            a.nps_score,
            CASE 
               WHEN a.nps_score >= 9 THEN 'promoters'
               WHEN a.nps_score < 7 THEN 'detractors'
               ELSE 'passives'
            END AS nps_group,
            b.country,
            b.city,
            SUBSTRING(c.title, 4) AS age_segment,
            SUBSTRING(d.title, 4) AS traffic_segment,
            SUBSTRING(e.title, 4) AS lifetime_segment
    FROM user a
        JOIN location b
        USING(location_id)
        JOIN age_segment c
        USING(age_gr_id)
        JOIN traffic_segment d
        USING(tr_gr_id)
        JOIN lifetime_segment e
        USING(lt_gr_id)
    WHERE b.country = 'Россия';
"""

In [6]:
df = pd.read_sql(query, engine)
df.sample(15)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,nps_score,nps_group,country,city,age_segment,traffic_segment,lifetime_segment
334914,KWVIAH,548,old,23.0,male,ANDROID,SMARTPHONE,10,promoters,Россия,РостовнаДону,16-24,30-35,13-24
43685,QNNC3Y,1484,old,51.0,female,ANDROID,SMARTPHONE,10,promoters,Россия,Волжский,45-54,50-55,36+
192324,H6CEEE,150,new,57.0,female,IOS,SMARTPHONE,5,detractors,Россия,Москва,55-64,1-5,4-6
190101,GB87D6,963,old,28.0,male,ANDROID,SMARTPHONE,7,passives,Россия,Москва,25-34,30-35,25-36
378379,NHY0UM,5259,old,52.0,female,ANDROID,SMARTPHONE,1,detractors,Россия,СанктПетербург,45-54,15-20,36+
354121,L7K881,290,new,38.0,male,ANDROID,SMARTPHONE,10,promoters,Россия,Самара,35-44,25-30,7-12
20315,PCEP1B,1304,old,50.0,female,ANDROID,SMARTPHONE,1,detractors,Россия,Белгород,45-54,5-10,36+
236166,YGE6A2,61,new,45.0,male,ANDROID,SMARTPHONE,5,detractors,Россия,Москва,45-54,10-15,3
17298,V6KMO0,952,old,63.0,female,ANDROID,SMARTPHONE,10,promoters,Россия,Барнаул,55-64,1-5,25-36
139217,LAWJCJ,5572,old,50.0,female,ANDROID,SMARTPHONE,2,detractors,Россия,Краснодар,45-54,15-20,36+


In [8]:
df.to_csv(r'C:\Users\System_10\Desktop\DA+\Python Projects\telecomm_csi_tableau.csv', index=False)

In [9]:
#Шаг 3-4 в Tableau:

Ссылка на презентацию:

https://public.tableau.com/views/_Komisarenko/_?:language=en-US&:sid=&:display_count=n&:origin=viz_share_link